In [1]:
from ast import literal_eval
from os import listdir
from os.path import isfile, join
from scipy.sparse import csr_matrix, load_npz, save_npz
from tqdm import tqdm
from sklearn.preprocessing import normalize

import seaborn as sns
import datetime
import json
import numpy as np
import pandas as pd
import time
import yaml
import scipy.sparse as sparse
from ast import literal_eval

# For Python2 this have to be done
from __future__ import division

In [2]:
# Load Original Data
df_train = pd.read_csv('../../data/beer/Train.csv')
df_valid = pd.read_csv('../../data/beer/Valid.csv')
df_test = pd.read_csv('../../data/beer/Test.csv')
key_phrase = pd.read_csv('../../data/beer/KeyPhrases.csv')

In [3]:
# Load U-I Data 
rtrain = load_npz("../../data/beer/Rtrain.npz")
# rvalid = load_npz("../../data/beer/Rvalid.npz")
# rtest = load_npz("../../data/beer/Rtest.npz")

In [4]:
rtrain

<6370x3668 sparse matrix of type '<type 'numpy.float32'>'
	with 167511 stored elements in Compressed Sparse Row format>

In [19]:
# Generate U_K and I_K
# For validation set
U_K_valid = get_I_K(df_valid, row_name = 'UserIndex', shape = (6370, 75))
I_K_valid = get_I_K(df_valid, row_name = 'ItemIndex', shape = (3668, 75))
# For test set
U_K_test = get_I_K(df_test, row_name = 'UserIndex', shape = (6370, 75))
I_K_test = get_I_K(df_test, row_name = 'ItemIndex', shape = (3668, 75))

100%|███████████████████████████████████████████████████████████████████████| 200546/200546 [00:10<00:00, 19498.88it/s]


In [23]:
# Save
save_npz('../../data/beer/U_K_valid.npz',U_K_valid)
save_npz('../../data/beer/I_K_valid.npz',I_K_valid)
save_npz( '../../data/beer/U_K_test.npz',U_K_test)
save_npz('../../data/beer/I_K_test.npz',I_K_test)

In [5]:
# Load 
U_K = load_npz('../../data/beer/U_K.npz')
I_K = load_npz('../../data/beer/I_K.npz')
U_K_test = load_npz('../../data/beer/U_K_test.npz')
I_K_test = load_npz('../../data/beer/I_K_test.npz')

In [6]:
df_valid.head()

,beer/ABV,beer/name,beer/style,review/overall,review/text,review/timeUnix,user/profileName,Binary,UserIndex,ItemIndex,review,conca_review,keyVector,keyphrases_indices_length
0,6.0,Rodenbach Grand Cru,Flanders Red Ale,5.0,"750ml corked, caged, & foiled bottle. Pretty. ...",1210818510,0110x011,1,0,2615,"['corked', 'caged', 'foiled', 'bottle', 'prett...",corked caged foiled bottle pretty pours clear...,"[1, 2, 14, 30, 36, 38, 41, 47, 58, 61, 62, 67]",12
1,10.0,Adam,Old Ale,4.0,"A - A dark brown bordering on black, and compl...",1237435182,0110x011,0,0,85,"['dark', 'brown', 'bordering', 'black', 'compl...",dark brown bordering black completely opaque ...,"[6, 14, 20, 21, 22, 31, 43, 44, 50, 54, 55, 63]",12
2,7.3,Green Flash West Coast I.P.A.,American IPA,4.0,"A - pours a hazy, copper/gold. Thick, chunky h...",1241393157,0110x011,0,0,1347,"['pours', 'hazy', 'copper', 'gold', 'thick', '...",pours hazy copper gold thick chunky head fade...,"[5, 12, 14, 31, 34, 40, 45, 47]",8
3,8.7,Stone Sublimely Self-Righteous Ale,American Black Ale,5.0,"Easily my favorite Stone beer, if not my all-t...",1208211124,0110x011,1,0,3147,"['easily', 'favorite', 'stone', 'beer', 'time'...",easily favorite stone beer time favorite beer...,"[0, 2, 6, 8, 13, 14, 16, 43, 50, 56, 60]",11
4,4.8,Calabaza Blanca,Witbier,4.5,Pours a cloudy straw yellow into a snifter. Bu...,1215571409,0110x011,1,0,657,"['pours', 'cloudy', 'straw', 'yelloww', 'snift...",pours cloudy straw yelloww snifter bubbly one...,"[14, 21, 30, 36, 37, 40, 51, 52]",8


In [7]:
# Models
from sklearn.metrics.pairwise import cosine_similarity
def train(matrix_train):
    similarity = cosine_similarity(X=matrix_train, Y=None, dense_output=True)
    return similarity

def get_I_K(df, row_name = 'ItemIndex', shape = (3668,75)):
    rows = []
    cols = []
    vals = []
    for i in tqdm(range(df.shape[0])):
        key_vector = literal_eval(df['keyVector'][i])
        rows.extend([df[row_name][i]]*len(key_vector)) ## Item index
        cols.extend(key_vector) ## Keyword Index
#         if binary:
        vals.extend(np.array([1]*len(key_vector)))
#         else:
#             vals.extend(arr[arr.nonzero()])    
    return csr_matrix((vals, (rows, cols)), shape=shape)


def predict(matrix_train, k, similarity, item_similarity_en = False):
    """
    res = similarity * matrix_train    if item_similarity_en = False
    res = similarity * matrix_train.T  if item_similarity_en = True
    """
    prediction_scores = []
    
    if item_similarity_en:
        matrix_train = matrix_train.transpose()
        
    for user_index in tqdm(range(matrix_train.shape[0])):
        # Get user u's prediction scores to all users
        vector_u = similarity[user_index]

        # Get closest K neighbors excluding user u self
        similar_users = vector_u.argsort()[::-1][1:k+1]
        # Get neighbors similarity weights and ratings
        similar_users_weights = similarity[user_index][similar_users]
        similar_users_ratings = matrix_train[similar_users].toarray()

        prediction_scores_u = similar_users_ratings * similar_users_weights[:, np.newaxis]

        prediction_scores.append(np.sum(prediction_scores_u, axis=0))
    res = np.array(prediction_scores)
    
    if item_similarity_en:
        res = res.transpose()
    
    return res

def prediction(prediction_score, topK, matrix_Train):

    prediction = []

    for user_index in tqdm(range(matrix_Train.shape[0])):
        vector_u = prediction_score[user_index]
        vector_train = matrix_Train[user_index]
        if len(vector_train.nonzero()[0]) > 0:
            vector_predict = sub_routine(vector_u, vector_train, topK=topK)
        else:
            vector_predict = np.zeros(topK, dtype=np.float32)

        prediction.append(vector_predict)

    return np.vstack(prediction)


def sub_routine(vector_u, vector_train, topK=500):

    train_index = vector_train.nonzero()[1]

    vector_u = vector_u

    candidate_index = np.argpartition(-vector_u, topK+len(train_index))[:topK+len(train_index)]
    vector_u = candidate_index[vector_u[candidate_index].argsort()[::-1]]
    vector_u = np.delete(vector_u, np.isin(vector_u, train_index).nonzero()[0])

    return vector_u[:topK]


In [8]:
# Evluation 
def recallk(vector_true_dense, hits, **unused):
    hits = len(hits.nonzero()[0])
    return float(hits)/len(vector_true_dense)

def precisionk(vector_predict, hits, **unused):
    hits = len(hits.nonzero()[0])
    return float(hits)/len(vector_predict)


def average_precisionk(vector_predict, hits, **unused):
    precisions = np.cumsum(hits, dtype=np.float32)/range(1, len(vector_predict)+1)
    return np.mean(precisions)


def r_precision(vector_true_dense, vector_predict, **unused):
    vector_predict_short = vector_predict[:len(vector_true_dense)]
    hits = len(np.isin(vector_predict_short, vector_true_dense).nonzero()[0])
    return float(hits)/len(vector_true_dense)


def _dcg_support(size):
    arr = np.arange(1, size+1)+1
    return 1./np.log2(arr)


def ndcg(vector_true_dense, vector_predict, hits):
    idcg = np.sum(_dcg_support(len(vector_true_dense)))
    dcg_base = _dcg_support(len(vector_predict))
    dcg_base[np.logical_not(hits)] = 0
    dcg = np.sum(dcg_base)
    return dcg/idcg


def click(hits, **unused):
    first_hit = next((i for i, x in enumerate(hits) if x), None)
    if first_hit is None:
        return 5
    else:
        return first_hit/10


def evaluate(matrix_Predict, matrix_Test, metric_names =['R-Precision', 'NDCG', 'Precision', 'Recall', 'MAP'], atK = [5, 10, 15, 20, 50], analytical=False):
    """
    :param matrix_U: Latent representations of users, for LRecs it is RQ, for ALSs it is U
    :param matrix_V: Latent representations of items, for LRecs it is Q, for ALSs it is V
    :param matrix_Train: Rating matrix for training, features.
    :param matrix_Test: Rating matrix for evaluation, true labels.
    :param k: Top K retrieval
    :param metric_names: Evaluation metrics
    :return:
    """
    global_metrics = {
        "R-Precision": r_precision,
        "NDCG": ndcg,
        "Clicks": click
    }

    local_metrics = {
        "Precision": precisionk,
        "Recall": recallk,
        "MAP": average_precisionk
    }

    output = dict()

    num_users = matrix_Predict.shape[0]

    for k in atK:

        local_metric_names = list(set(metric_names).intersection(local_metrics.keys()))
        results = {name: [] for name in local_metric_names}
        topK_Predict = matrix_Predict[:, :k]

        for user_index in tqdm(range(topK_Predict.shape[0])):
            vector_predict = topK_Predict[user_index]
            if len(vector_predict.nonzero()[0]) > 0:
                vector_true = matrix_Test[user_index]
                vector_true_dense = vector_true.nonzero()[1]
                hits = np.isin(vector_predict, vector_true_dense)

                if vector_true_dense.size > 0:
                    for name in local_metric_names:
                        results[name].append(local_metrics[name](vector_true_dense=vector_true_dense,
                                                                 vector_predict=vector_predict,
                                                                 hits=hits))

        results_summary = dict()
        if analytical:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = results[name]
        else:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = (np.average(results[name]),
                                                              1.96*np.std(results[name])/np.sqrt(num_users))
        output.update(results_summary)

    global_metric_names = list(set(metric_names).intersection(global_metrics.keys()))
    results = {name: [] for name in global_metric_names}

    topK_Predict = matrix_Predict[:]

    for user_index in tqdm(range(topK_Predict.shape[0])):
        vector_predict = topK_Predict[user_index]

        if len(vector_predict.nonzero()[0]) > 0:
            vector_true = matrix_Test[user_index]
            vector_true_dense = vector_true.nonzero()[1]
            hits = np.isin(vector_predict, vector_true_dense)

            # if user_index == 1:
            #     import ipdb;
            #     ipdb.set_trace()

            if vector_true_dense.size > 0:
                for name in global_metric_names:
                    results[name].append(global_metrics[name](vector_true_dense=vector_true_dense,
                                                              vector_predict=vector_predict,
                                                              hits=hits))

    results_summary = dict()
    if analytical:
        for name in global_metric_names:
            results_summary[name] = results[name]
    else:
        for name in global_metric_names:
            results_summary[name] = (np.average(results[name]), 1.96*np.std(results[name])/np.sqrt(num_users))
    output.update(results_summary)

    return output

def explain_evaluate(matrix_Predict, matrix_Test, metric_names =['R-Precision', 'NDCG', 'Precision', 'Recall', 'MAP'], atK = [5, 10, 15, 20, 50], analytical=False):
    """
    :param matrix_U: Latent representations of users, for LRecs it is RQ, for ALSs it is U
    :param matrix_V: Latent representations of items, for LRecs it is Q, for ALSs it is V
    :param matrix_Train: Rating matrix for training, features.
    :param matrix_Test: Rating matrix for evaluation, true labels.
    :param k: Top K retrieval
    :param metric_names: Evaluation metrics
    :return:
    """
    global_metrics = {
        "R-Precision": r_precision,
        "NDCG": ndcg,
        "Clicks": click
    }

    local_metrics = {
        "Precision": precisionk,
        "Recall": recallk,
        "MAP": average_precisionk
    }

    output = dict()

    num_users = matrix_Predict.shape[0]

    for k in atK:

        local_metric_names = list(set(metric_names).intersection(local_metrics.keys()))
        results = {name: [] for name in local_metric_names}
        topK_Predict = matrix_Predict[:, :k]

        for user_index in tqdm(range(topK_Predict.shape[0])):
            vector_predict = topK_Predict[user_index]
            if len(vector_predict.nonzero()[0]) > 0:
#                 vector_true = matrix_Test[user_index]
                vector_true = np.ravel(matrix_Test.todense()[0])
                vector_true_dense = np.argsort(vector_true)[::-1][:k]
#                 vector_true_dense = vector_true.nonzero()[1]
                hits = np.isin(vector_predict, vector_true_dense)

                if vector_true_dense.size > 0:
                    for name in local_metric_names:
                        results[name].append(local_metrics[name](vector_true_dense=vector_true_dense,
                                                                 vector_predict=vector_predict,
                                                                 hits=hits))

        results_summary = dict()
        if analytical:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = results[name]
        else:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = (np.average(results[name]),
                                                              1.96*np.std(results[name])/np.sqrt(num_users))
        output.update(results_summary)

    global_metric_names = list(set(metric_names).intersection(global_metrics.keys()))
    results = {name: [] for name in global_metric_names}

    topK_Predict = matrix_Predict[:]

    for user_index in tqdm(range(topK_Predict.shape[0])):
        vector_predict = topK_Predict[user_index]

        if len(vector_predict.nonzero()[0]) > 0:
#             vector_true = matrix_Test[user_index]
#             vector_true_dense = vector_true.nonzero()[1]
            vector_true = np.ravel(matrix_Test.todense()[0])
            vector_true_dense = np.argsort(vector_true)[::-1][:k]
            hits = np.isin(vector_predict, vector_true_dense)

            # if user_index == 1:
            #     import ipdb;
            #     ipdb.set_trace()

            if vector_true_dense.size > 0:
                for name in global_metric_names:
                    results[name].append(global_metrics[name](vector_true_dense=vector_true_dense,
                                                              vector_predict=vector_predict,
                                                              hits=hits))

    results_summary = dict()
    if analytical:
        for name in global_metric_names:
            results_summary[name] = results[name]
    else:
        for name in global_metric_names:
            results_summary[name] = (np.average(results[name]), 1.96*np.std(results[name])/np.sqrt(num_users))
    output.update(results_summary)

    return output

In [101]:
np.ravel(U_K_valid.todense()[0])

array([ 1,  7,  7,  0,  1,  3,  3,  0,  2,  0,  0,  0,  6,  3, 10,  1,  3,
        0,  1,  0,  3,  5,  1,  1,  3,  0,  2,  2,  0,  1,  2,  5,  0,  0,
        2,  0,  4,  1,  2,  2,  3,  2,  3,  6,  5,  1,  0,  5,  0,  0,  4,
        2,  1,  1,  1,  2,  1,  0,  2,  0,  3,  1,  1,  2,  1,  0,  2,  3,
        2,  0,  0,  1,  0,  0,  1])

In [103]:
np.argsort(np.ravel(U_K_valid.todense()[0]))[::-1][:20]

array([14,  1,  2, 43, 12, 44, 21, 31, 47, 50, 36, 20, 42, 16, 40, 13, 60,
        6, 67,  5], dtype=int64)

# Explanation Model

Explanation could be a list of similar items or stated reasons in natural language. Keyphrases mined from user reviews are used here to reflect reasons for user like or dislike an item. 

In [9]:
def explain(R,W2,k, model = "Cosine_similarity", item_similarity_en = True):
    """
    k: knn's hyperparameter k
    R: Rating Matrix with size U*I
    r_ij: observed rating with user i and item j 
    s_ij: explanation vector with user i and item j 
    Z: Joint Embedding/Latent Space with size U*U, generate r_ij and s_ij
    W2: Reconstruction matrix with size U*K 
    S: Output explanation prediction matrix with size U*K (dense numpy ndarray)
    """
    Z = train(R) # Cosine similarity as default
    S = predict(W2, k, Z, item_similarity_en=item_similarity_en) 
    if normalize_en == True:       
        return normalize(S) # prediction score
    return S

def predict(matrix_train, k, similarity, item_similarity_en = False, normalize_en = False):
    """
    matrix_train: Rating Matrix with size U*I
    k: knn's hyperparameter k
    similarity: Joint Embedding/Latent Space with size U*U or I*I
    
    res = similarity * matrix_train    if item_similarity_en = False
    res = similarity * matrix_train.T  if item_similarity_en = True
    
    r_ij: observed rating with user i and item j 
    s_ij: explanation vector with user i and item j 
    """
    prediction_scores = []
    
    if item_similarity_en:
        matrix_train = matrix_train.transpose()
        
    for user_index in tqdm(range(matrix_train.shape[0])):
        # Get user u's prediction scores to all users
        vector_u = similarity[user_index]

        # Get closest K neighbors excluding user u self
        similar_users = vector_u.argsort()[::-1][1:k+1]
        # Get neighbors similarity weights and ratings
        similar_users_weights = similarity[user_index][similar_users]
        similar_users_ratings = matrix_train[similar_users].toarray()

        prediction_scores_u = similar_users_ratings * similar_users_weights[:, np.newaxis]

        prediction_scores.append(np.sum(prediction_scores_u, axis=0))
    res = np.array(prediction_scores)
    
    if item_similarity_en:
        res = res.transpose()
    if normalize_en:
        res = normalize(res)
    return res

def explain_prediction(prediction_score, topK, matrix_Train):
    """
    output prediction res of the  top K items/keyphrase/whatever
    """
    prediction = []

    for user_index in tqdm(range(matrix_Train.shape[0])):
        vector_u = prediction_score[user_index]
        vector_train = matrix_Train[user_index]
        if len(vector_train.nonzero()[0]) > 0:
            vector_predict = sub_routine_explain(vector_u, vector_train, topK=topK)
        else:
            vector_predict = np.zeros(topK, dtype=np.float32)

        prediction.append(vector_predict)
    return np.vstack(prediction)
#     return prediction

def sub_routine_explain(vector_u, vector_train, topK=30):
    """
    vector_u: predicted user vector
    vector_train: true user vector
    topK: top k items in vector
    vector_u: top k items predicted
    """
#     train_index = vector_train.nonzero()[1]
#     candidate_index = np.argpartition(-vector_u, topK+75)[:topK+75] #  10 here to make res consistent
#     candidate_index = np.argpartition(-vector_u, 74)[:topK]
#     vector_u = candidate_index[vector_u[candidate_index].argsort()[::-1]]
#     vector_u = np.delete(vector_u, np.isin(vector_u, train_index).nonzero()[0])

    candidate_index = np.argsort(vector_u)[::-1][:topK]
    return candidate_index

def predict_pilot_explanation(explanation_scores, top_keyphrase = 10):
    """
    Used for retrieve the 1st row of prediction scores, used for pilot test
    """
    explanation = []
    for explanation_score in tqdm(explanation_scores):
        explanation.append(np.argsort(explanation_score)[::-1][:top_keyphrase])
    return np.array(explanation)

In [10]:
def explain(R,W2,k):
    """
    R: Rating Matrix with size U*I
    r_ij: observed rating with user i and item j 
    s_ij: explanation vector with user i and item j 
    Z: Joint Embedding/Latent Space with size U*U, generate r_ij and s_ij
    W2: Reconstruction matrix with size U*K 
    S: Output explanation prediction matrix with size U*K
    """
    Z = train(R)
    S = predict(W2, k, Z)
    return normalize(S)

In [11]:
# Evaluation Model
def recall(vector_true_dense, vector_true_predict):
    """
    The fraction of relevant instances that have been retrieved over the total amount of relevant instances
    The length of vector_true_dense and vector_true_predict has to be the same
    Out put recall
    """
    hits = len(np.isin(vector_true_predict, vector_true_dense).nonzero()[0])
    return float(hits)/len(vector_true_dense)

# Total Recall
def recall_all(true_matrix, predict_matrix, topK = 20):
    res = []
    for i in tqdm(range(len(Explanation_res1))):
        true_vector = np.argsort(np.ravel(normalize(true_matrix).todense()[i]))[-topK:]
        predict_vector = np.argsort(predict_matrix[i])[-topK:]
        res.append(recall(true_vector,predict_vector))
    return sum(res)/len(res)

In [12]:
# ( U_I * I_U ) * U_K
similarity = train(rtrain)
explanation_scores = predict(U_K, 100, similarity)
# explanation =  predict_explanation(explanation_scores)
explanation = explain_prediction(explanation_scores, 70, U_K)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:00<00:00, 8516.04it/s]


# Evaluation Model

In [10]:
explain_evaluate(explanation, U_K_test, atK=[5,10,20])

100%|█████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:33<00:00, 189.78it/s]


{'MAP@10': (0.88274159004261032, 0.0017655296910119374),
 'MAP@20': (0.83116420958261306, 0.0012303964162601892),
 'MAP@5': (0.69733699633699642, 0.0019366605097710626),
 'NDCG': (0.91778672272069384, 0.00069386408012016905),
 'Precision@10': (0.72006279434850851, 0.0017404610660916515),
 'Precision@20': (0.64006279434850877, 0.0011064512227747327),
 'Precision@5': (0.60919937205651487, 0.0025503389596737247),
 'R-Precision': (0.64006279434850877, 0.0011064512227747327),
 'Recall@10': (0.72006279434850851, 0.0017404610660916515),
 'Recall@20': (0.64006279434850877, 0.0011064512227747327),
 'Recall@5': (0.60919937205651487, 0.0025503389596737247)}

# Tuning

TODO

# Item-based non-personalized Explanation

In [11]:
similarity = train(np.transpose(rtrain))
explanation_scores = predict(I_K, 100, similarity)
# explanation =  predict_explanation(explanation_scores)
explanation = explain_prediction(explanation_scores, 70, I_K)

100%|████████████████████████████████████████████████████████████████████████████| 3668/3668 [00:01<00:00, 2729.17it/s]


In [150]:
explain_evaluate(explanation, I_K_valid, atK=[5,10,20]) 

100%|████████████████████████████████████████████████████████████████████████████| 3668/3668 [00:02<00:00, 1368.66it/s]


{'MAP@10': (0.7280294178740199, 0.003719951721807636),
 'MAP@20': (0.8368168788179973, 0.001992342237049055),
 'MAP@5': (0.5433960378044348, 0.0029654908999895008),
 'NDCG': (0.9257757747057292, 0.0008229360296267417),
 'Precision@10': (0.6539531079607415, 0.004636637273117065),
 'Precision@20': (0.6580834242093784, 0.0018566550515704686),
 'Precision@5': (0.4323336968375136, 0.005050260732533747),
 'R-Precision': (0.6580834242093784, 0.0018566550515704686),
 'Recall@10': (0.6539531079607415, 0.004636637273117065),
 'Recall@20': (0.6580834242093784, 0.0018566550515704686),
 'Recall@5': (0.4323336968375136, 0.005050260732533747)}

In [12]:
explain_evaluate(explanation, I_K_test, atK=[5,10,20]) 

100%|█████████████████████████████████████████████████████████████████████████████| 3668/3668 [00:16<00:00, 225.72it/s]


{'MAP@10': (0.72815139732737888, 0.0037163059650557467),
 'MAP@20': (0.74832825993365049, 0.0031152175169084509),
 'MAP@5': (0.54312704471101414, 0.0029756141294349702),
 'NDCG': (0.89379167048593566, 0.0011602119212643241),
 'Precision@10': (0.65406215921483091, 0.0046303761758466466),
 'Precision@20': (0.54847328244274807, 0.0022367207717170329),
 'Precision@5': (0.43244274809160305, 0.0050472658635987242),
 'R-Precision': (0.54847328244274807, 0.0022367207717170329),
 'Recall@10': (0.65406215921483091, 0.0046303761758466466),
 'Recall@20': (0.54847328244274807, 0.0022367207717170329),
 'Recall@5': (0.43244274809160305, 0.0050472658635987242)}

# TF-IDF 

In [192]:
def df(U_K):
    """
    Get the df for keyphrase matrix
    """
    DF = {}
    for i in range(U_K.shape[0]):
        vector_u = np.ravel(U_K[i].todense())
        for keyword_idx in range(U_K.shape[1]):
            keyword_frequency = vector_u[keyword_idx]
            if keyword_idx in DF:
                DF[keyword_idx] += keyword_frequency
            else:
                DF[keyword_idx] = keyword_frequency
    return DF

def keyword_popularity_matrix(U_K,df_uk):
    """
    return the U_K normalized by keyphrase frequency
    """
    df = normalize(np.array(df_uk.values()).reshape(1,-1))[0]
    for i in tqdm(range(U_K.shape[0])):
        U_K[i] = U_K[i]/df
#         vector_u = np.ravel(U_K[i].todense())
#         for entry in range(U_K.shape[1]):
#             U_K[i,entry] = U_K[i,entry]/df[entry]
    return normalize(U_K)
    
def tf_idf(U_K,df_uk):
    """
    Change the U_K/I_K keyphrase matrix to TF-IDF.
    where:
    IDF = log(N/(DF+1))
    TF_IDF = (1+log(TF))* IDF
    """
    for i in tqdm(range(U_K.shape[0])):
        vector_u = np.ravel(U_K[i].todense())
        words_count = np.sum(vector_u)
        
        for entry in range(U_K.shape[1]):
            if U_K[i,entry] == 0:
                continue
            else:
                tf = U_K[i,entry]/words_count
                df = df_uk[entry]
                idf = np.log(U_K.shape[0]/(df+1))
                tf_idf = (1+ np.log(tf))*idf
                U_K[i,entry] = tf_idf
    return normalize(U_K)

In [185]:
df = normalize(np.array(df_uk.values()).reshape(1,-1))[0]

In [190]:
U_K[0] = U_K[0]/df

In [ ]:
U_K_pop=keyword_popularity_matrix(U_K,df_uk) 

In [195]:
U_K_test_pop=keyword_popularity_matrix(U_K_test,df_uk) 

100%|█████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:57<00:00, 110.77it/s]


In [189]:
# Load 
U_K = load_npz('../../data/beer/U_K.npz')
I_K = load_npz('../../data/beer/I_K.npz')
U_K_test = load_npz('../../data/beer/U_K_test.npz')
I_K_test = load_npz('../../data/beer/I_K_test.npz')

In [20]:
df_uk = df(U_K)

### Example of  TF-IDF vs. TF

In [90]:
U_K[0].todense() # TF

matrix([[ 7, 35, 27,  1,  6,  5, 13,  6,  9,  0,  4,  7, 13,  6, 44,  1, 20,
          4,  7,  2,  7, 16,  8,  1,  4,  0, 11,  3,  0,  1, 16, 14,  4,  3,
          9,  2, 19,  2,  3, 13, 10,  5,  6, 22, 17,  5,  0, 11,  1,  1, 13,
          4,  1,  5,  4, 10,  4,  0, 15,  0,  7,  4, 12, 10,  7,  2,  1, 15,
          2,  0,  6,  0,  2,  2,  0]])

In [88]:
U_K[0].todense() # TF-IDF

matrix([[ 9,  6,  6,  6,  7,  9,  8,  5,  7,  0,  9,  8,  8,  6,  5,  6,  8,
          5,  8,  8,  6,  7,  5,  7,  9,  0,  5,  5,  0,  6,  5,  5,  5,  2,
          6,  5,  5,  0,  7,  8,  6,  8, 11,  7,  8, 10,  0,  6,  3,  0,  7,
          8,  7,  3,  2,  9,  6,  0,  6,  0,  5,  6,  5,  3,  4,  7,  5,  4,
          9,  0,  1,  0,  3,  3,  0]])

In [114]:
U_K_tfidf = tf_idf(U_K, df_uk)

100%|█████████████████████████████████████████████████████████████████████████████| 6370/6370 [01:01<00:00, 103.26it/s]


In [95]:
similarity = train(rtrain)
explanation_scores = predict(U_K_tfidf, 100, similarity)
# explanation =  predict_explanation(explanation_scores)
explanation = explain_prediction(explanation_scores, 70, U_K_tfidf)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:02<00:00, 2776.81it/s]


In [99]:
df_uk_test = df(U_K_test)

In [100]:
U_K_test_tfidf = tf_idf(U_K_test, df_uk_test)

100%|█████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:47<00:00, 131.01it/s]


In [101]:
explain_evaluate(explanation, U_K_test_tfidf, atK=[5,10,20])

100%|█████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:31<00:00, 202.97it/s]


{'MAP@10': (0.31673970247439637, 0.0040957390087849946),
 'MAP@20': (0.72956795987541523, 0.0019829474684527073),
 'MAP@5': (0.14120355834641549, 0.0043018567976726993),
 'NDCG': (0.88171247369908323, 0.0013367377062992105),
 'Precision@10': (0.37629513343799059, 0.0023021588046979805),
 'Precision@20': (0.75726059654631084, 0.0014119254522028722),
 'Precision@5': (0.15871271585557301, 0.0035101400942391058),
 'R-Precision': (0.75726059654631084, 0.0014119254522028722),
 'Recall@10': (0.37629513343799059, 0.0023021588046979805),
 'Recall@20': (0.75726059654631084, 0.0014119254522028722),
 'Recall@5': (0.15871271585557301, 0.0035101400942391058)}

# TF-pop 

In [194]:
similarity = train(rtrain)
explanation_scores = predict(U_K_pop, 100, similarity)
# explanation =  predict_explanation(explanation_scores)
explanation = explain_prediction(explanation_scores, 70, U_K_pop)

100%|████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:01<00:00, 5295.10it/s]


In [196]:
explain_evaluate(explanation, U_K_test_pop, atK=[5,10,20])

100%|█████████████████████████████████████████████████████████████████████████████| 6370/6370 [00:43<00:00, 146.66it/s]


{'MAP@10': (0.13406489995265505, 0.0042720301256862969),
 'MAP@20': (0.18142879311319321, 0.0038381696109094855),
 'MAP@5': (0.11403558346415489, 0.0053400341804486703),
 'NDCG': (0.56953074240666524, 0.0023574928609448264),
 'Precision@10': (0.09651491365777079, 0.0026611850558291295),
 'Precision@20': (0.16945839874411303, 0.0026470594420987216),
 'Precision@5': (0.077739403453689182, 0.0034150165762856983),
 'R-Precision': (0.16945839874411303, 0.0026470594420987216),
 'Recall@10': (0.09651491365777079, 0.0026611850558291295),
 'Recall@20': (0.16945839874411303, 0.0026470594420987216),
 'Recall@5': (0.077739403453689182, 0.0034150165762856983)}

# TF-IDF for Item-based explanation

In [104]:
df_ik = df(I_K)

In [106]:
I_K_tfidf = tf_idf(I_K,df_ik)

100%|██████████████████████████████████████████████████████████████████████████████| 3668/3668 [00:40<00:00, 91.11it/s]


In [107]:
df_ik_test = df(I_K_test)

In [108]:
I_K_test_tfidf = tf_idf(I_K_test,df_ik_test)

100%|█████████████████████████████████████████████████████████████████████████████| 3668/3668 [00:31<00:00, 117.52it/s]


In [109]:
similarity = train(np.transpose(rtrain))
explanation_scores = predict(I_K_tfidf, 100, similarity)
# explanation =  predict_explanation(explanation_scores)
explanation = explain_prediction(explanation_scores, 70, I_K_tfidf)

100%|████████████████████████████████████████████████████████████████████████████| 3668/3668 [00:00<00:00, 6013.11it/s]


In [110]:
explain_evaluate(explanation, I_K_test_tfidf, atK=[5,10,20])

100%|█████████████████████████████████████████████████████████████████████████████| 3668/3668 [00:10<00:00, 337.44it/s]


{'MAP@10': (0.28507111050527084, 0.0047971981576525376),
 'MAP@20': (0.5359339768182384, 0.0042830473646057639),
 'MAP@5': (0.11980370774263904, 0.0055372076435837875),
 'NDCG': (0.79679938734343247, 0.0022304167988526492),
 'Precision@10': (0.37753544165757913, 0.0038051249864415521),
 'Precision@20': (0.61166848418756825, 0.0031038667844431093),
 'Precision@5': (0.13718647764449293, 0.0049324913227143715),
 'R-Precision': (0.61166848418756825, 0.0031038667844431093),
 'Recall@10': (0.37753544165757913, 0.0038051249864415521),
 'Recall@20': (0.61166848418756825, 0.0031038667844431093),
 'Recall@5': (0.13718647764449293, 0.0049324913227143715)}

# Create a synthetic user u1

### Variables
- keyphrase_ids: list of 0~75, keyphrases that synthetic user like
- top_n: number of resturants associated with keyword 
- k: knn hyperparameter

In [132]:
# Get top_items items associated with keyphrase_ids 
def item_associated_with_keyphrase(I_K, keyphrase_ids, top_items = 100):
    """
    I_K: Item Keyphrase Matrix
    Keyphrase_ids: top items described by keyphrase_ids
    top_n: how many top items described by the keyphrases will be output 
    output item list (unique)
    """
    res = []
    for keyphrase_id in keyphrase_ids:
        res.append(np.argsort(np.ravel(I_K.todense()[:,keyphrase_id]))[::-1][:top_items])
    return np.unique(res)

# Modify U_U latent Space from U_I
def modify_user_preference(U_I, items, user_id = 0):
    """
    TODO: Fix the function s.t. it will not modify the initial matrix
    """
    U_I[user_id,:] = 0
    for i in items:
        U_I[0,i] = 1
    return U_I

def clear_user_keyphrase(U_K, user_id = 0):
    U_K[user_id,:] = 0

def explain_synthetic_user(rtrain, U_K, I_K, keyphrases, top_keyphrase = 20, user_id = 0, k = 100, top_items = 100, **Not_used):
    """
    
    """
    items = item_associated_with_keyphrase(I_K, keyphrases, top_items = top_items) # 8 is coffee
    U_I = modify_user_preference(rtrain, items, user_id=user_id)
    modified_U_U = train(U_I)
    U_K[user_id, :] = 0
    synthetic_user_keyphrase = normalize(predict(U_K, k, modified_U_U))[user_id]
    return np.argsort(synthetic_user_keyphrase)[::-1][:top_keyphrase]

def modify_user_keyphrase(U_K, keyphrase_ids, normalization = True, keyval = 1, user_id = 0, **Not_Used):
    """
    Change all keyphrase_ids to some fixed number, all others to 0
    Return the U_K matrix with user_id row the synthetic user1
    """
    U_K[user_id,:] = 0
    for keyphrase_id in keyphrase_ids:
        U_K[user_id,keyphrase_id] = keyval
    if normalization == True:
        return normalize(U_K)
    return U_K

In [8]:
# U_K = get_I_K(df_train, row_name = 'UserIndex', shape = (6370,75))
U_K = load_npz('../../data/beer/U_K.npz')

In [103]:
# I_K = get_I_K(df_train, row_name = 'ItemIndex', shape = (3668,75))
I_K = load_npz('../../data/beer/I_K.npz')

### Experiment

In [38]:
keyphrases = pd.read_csv("../../data/beer/KeyPhrases.csv")['Phrases'].tolist()

In [39]:
keyphrases

['roast',
 'sweet',
 'caramel',
 'backbone',
 'toast',
 'bread',
 'chocolate',
 'biscuit',
 'coffee',
 'wheat',
 'grain',
 'yeast',
 'citrus',
 'toffee',
 'fruit',
 'nutty',
 'bitter',
 'barley',
 'floral',
 'honey',
 'earthy',
 'spic',
 'smok',
 'molass',
 'sugar',
 ' rye ',
 'grass',
 'cocoa',
 'chew',
 'butter',
 'sour',
 ' pine ',
 'herbal',
 'citric',
 'grapefruit',
 'resin',
 'crisp',
 'pungent',
 'wood',
 'orang',
 'lemon',
 'offwhite',
 ' white',
 'tan',
 'brown',
 'gold',
 'beige',
 'copper',
 'mocha',
 'khaki',
 'black',
 'yellow',
 'watery',
 'slick',
 'silk',
 'smooth',
 'syrup',
 'velvet',
 ' red ',
 'mahogany',
 'ruby',
 'tulip',
 'cherry',
 'plum',
 'berry',
 'banana',
 'bourbon',
 'oak',
 'apple',
 'metallic',
 'raspberry',
 'clove',
 'apricot',
 'coriander',
 'pepper']

In [41]:
keyphrases[70]

'raspberry'

In [153]:
U_K = load_npz('../../data/beer/U_K.npz')
U_K = normalize(U_K)
rtrain = load_npz("../../data/beer/Rtrain.npz")

## Prediction pipline

In [140]:
# Modify user preference matrix
items = item_associated_with_keyphrase(I_K, [70], top_items = 200) # 'raspberry'
U_I = modify_user_preference(rtrain, items, user_id = 0)

D:\Anaconda\envs\py27\lib\site-packages\ipykernel_launcher.py:19: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.


In [141]:
# make synthetic user1's keyphrase preference all 0
clear_user_keyphrase(U_K, user_id = 0)

D:\Anaconda\envs\py27\lib\site-packages\ipykernel_launcher.py:25: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.


In [142]:
# Get latent user similarity embedding
modified_U_U = train(U_I)
# predict
explanation_scores1 = predict(U_K, 100, modified_U_U)
explanation1 =  predict_pilot_explanation(explanation_scores1, top_keyphrase = 10)

100%|██████████████████████████████████████████████████████████████████████████| 6370/6370 [00:00<00:00, 244999.78it/s]


In [144]:
explanation1[0]

array([ 1, 16, 14, 44, 43, 42, 55,  2, 21, 12], dtype=int64)

## Evaluate the pilot test result

Since we create a sterotype user1 who only likes keyphrase 0, and let him liked some beer with keyphrase 0, we can evaluate the result U_K (user1's updated keyphrase preferance) by looking into what's the top K keyphrase he likes other than keyphrase 0. Then we look into: among the top N (100 in this case) beer with keyphrase 0, whats the next K (e.g. 10) most frequent keyphrase that appear, and compare this with the updated_U_K to see if user1 actually likes the other K keyphrase as we predicted. 

In [146]:
def rank_in_prediction(rtrain, U_K, I_K, top_items = 200, keyphrase = [70],user_i = 0):
    """
    Get the rank for user_i with keyphrase
    TODO: modify so that no need to reload U_K,I_K
    """  
    U_K = load_npz('../../data/beer/U_K.npz')
    U_K = normalize(U_K)
    rtrain = load_npz("../../data/beer/Rtrain.npz")
    
    # Modify user preference matrix
    items = item_associated_with_keyphrase(I_K, keyphrase, top_items = top_items) # 'raspberry'
    U_I = modify_user_preference(rtrain, items, user_id = 0)
    
    # make synthetic user1's keyphrase preference all 0
    clear_user_keyphrase(U_K, user_id = 0)
    
    # Get latent user similarity embedding
    modified_U_U = train(U_I)
    # predict
    explanation_scores1 = predict(U_K, 100, modified_U_U)
    explanation1 =  predict_pilot_explanation(explanation_scores1, top_keyphrase = 75)
    return list(explanation1[user_i]).index(keyphrase[0])
    
def evaluate_pilot_test(rtrain,U_K, I_K,keyphrase_list, top_items = 200, user_i = 0):
    # Get the average rank for user_i with keyphrase  
    res1 = 0
    for i in range(75):
        a = rank_in_prediction(rtrain, U_K, I_K, top_items = top_items, keyphrase = [i],user_i = user_i)
        print "keyphrase", keyphrase_list[i], "'s rank is ", a
        res1+= a
    return res1/75

In [147]:
evaluate_pilot_test(rtrain,U_K,I_K,keyphrases)

D:\Anaconda\envs\py27\lib\site-packages\ipykernel_launcher.py:19: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
D:\Anaconda\envs\py27\lib\site-packages\ipykernel_launcher.py:25: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
100%|██████████████████████████████████████████████████████████████████████████| 6370/6370 [00:00<00:00, 235926.36it/s]


keyphrase roast 's rank is  8


100%|██████████████████████████████████████████████████████████████████████████| 6370/6370 [00:00<00:00, 244999.78it/s]


keyphrase sweet 's rank is  0


100%|██████████████████████████████████████████████████████████████████████████| 6370/6370 [00:00<00:00, 244999.78it/s]


...

100%|██████████████████████████████████████████████████████████████████████████| 6370/6370 [00:00<00:00, 227500.99it/s]


keyphrase coriander 's rank is  46


100%|██████████████████████████████████████████████████████████████████████████| 6370/6370 [00:00<00:00, 244999.78it/s]


keyphrase pepper 's rank is  30


28.026666666666667

In [148]:
keyphrases

['roast',
 'sweet',
 'caramel',
 'backbone',
 'toast',
 'bread',
 'chocolate',
 'biscuit',
 'coffee',
 'wheat',
 'grain',
 'yeast',
 'citrus',
 'toffee',
 'fruit',
 'nutty',
 'bitter',
 'barley',
 'floral',
 'honey',
 'earthy',
 'spic',
 'smok',
 'molass',
 'sugar',
 ' rye ',
 'grass',
 'cocoa',
 'chew',
 'butter',
 'sour',
 ' pine ',
 'herbal',
 'citric',
 'grapefruit',
 'resin',
 'crisp',
 'pungent',
 'wood',
 'orang',
 'lemon',
 'offwhite',
 ' white',
 'tan',
 'brown',
 'gold',
 'beige',
 'copper',
 'mocha',
 'khaki',
 'black',
 'yellow',
 'watery',
 'slick',
 'silk',
 'smooth',
 'syrup',
 'velvet',
 ' red ',
 'mahogany',
 'ruby',
 'tulip',
 'cherry',
 'plum',
 'berry',
 'banana',
 'bourbon',
 'oak',
 'apple',
 'metallic',
 'raspberry',
 'clove',
 'apricot',
 'coriander',
 'pepper']

# Create Synthetic Item i1

2 things can be done here, we already did 1, will do 2 this time.
1. Cold-start: we have a item1 which haven't been tried by any users, but we know its keyphrases I_K, we can predict who (users) will like it by constructing I_I and mutiply by I_U to get predicted I_U
2. We know item1 is liked by some user (those user have a certain taste described by U_K matrix), we can then describe/categorize it with some keyphrase I_K

In [167]:
# Get top_users items associated with keyphrase_ids 
def users_with_keyphrase_preference(U_K, keyphrase_ids, top_users = 100, norm = True):
    """
    U_K: User Keyphrase Matrix
    Keyphrase_ids: top_users who like keyphrase_ids
    output item list (unique)
    """
    res = []
    if norm:
        U_K = normalize(U_K)
    try:
        for keyphrase_id in keyphrase_ids:
            res.append(np.argsort(np.ravel(U_K.todense()[:,keyphrase_id]))[::-1][:top_users])
    except:
        return np.argsort(np.ravel(U_K.todense()[:,keyphrase_ids]))[::-1][:top_users]
    return np.unique(res)

# Modify I_I latent Space from U_I
def modify_item_history(U_I, users, item_id = 0):
    """
    TODO: Fix the function s.t. it will not modify the initial matrix
    """
    U_I[:,item_id] = 0
    for i in users:
        U_I[i, item_id] = 1
    return normalize(U_I)

def clear_item_keyphrase(I_K, item_id = 0):
    I_K[item_id, :] = 0

def explain_synthetic_item(rtrain, U_K, I_K, keyphrases, top_keyphrase = 20, user_id = 0, k = 100, top_users = 100, **Not_used):
    """
    
    """
    items = item_associated_with_keyphrase(I_K, keyphrases, top_items = top_items) # 8 is coffee
    U_I = modify_user_preference(rtrain, items, user_id=user_id)
    modified_U_U = train(U_I)
    U_K[user_id, :] = 0
    synthetic_user_keyphrase = normalize(predict(U_K, k, modified_U_U))[user_id]
    return np.argsort(synthetic_user_keyphrase)[::-1][:top_keyphrase]

def modify_user_keyphrase(U_K, keyphrase_ids, normalization = True, keyval = 1, user_id = 0, **Not_Used):
    """
    Change all keyphrase_ids to some fixed number, all others to 0
    Return the U_K matrix with user_id row the synthetic user1
    """
    U_K[user_id,:] = 0
    for keyphrase_id in keyphrase_ids:
        U_K[user_id,keyphrase_id] = keyval
    if normalization == True:
        return normalize(U_K)
    return U_K

## Prediction Pipeline

In [174]:
U_K = load_npz('../../data/beer/U_K.npz')
I_K = load_npz('../../data/beer/I_K.npz')
U_K = normalize(U_K)
I_K = normalize(I_K)
rtrain = load_npz("../../data/beer/Rtrain.npz")

In [169]:
# Modify user preference matrix
users = users_with_keyphrase_preference(U_K, 70, top_users = 100, norm = True) # 'raspberry'
U_I = modify_item_history(rtrain, users, item_id = 0)

D:\Anaconda\envs\py27\lib\site-packages\ipykernel_launcher.py:23: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.


In [170]:
# make synthetic item1's keyphrase to all 0
clear_item_keyphrase(I_K, item_id = 0)

D:\Anaconda\envs\py27\lib\site-packages\ipykernel_launcher.py:29: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.


In [179]:
# Get latent item similarity embedding
modified_I_I = train(np.transpose(U_I))
# predict
explanation_scores1 = predict(I_K, 100, modified_I_I)
explanation1 =  predict_pilot_explanation(explanation_scores1, top_keyphrase = 70)

100%|██████████████████████████████████████████████████████████████████████████| 3668/3668 [00:00<00:00, 215765.21it/s]


In [185]:
def rank_in_prediction_item(rtrain, U_K, I_K, top_users = 200, keyphrase = 70, item_i = 0):
    """
    Get the rank for user_i with keyphrase
    TODO: modify so that no need to reload U_K,I_K
    """  
    U_K = load_npz('../../data/beer/U_K.npz')
    I_K = load_npz('../../data/beer/I_K.npz')
    U_K = normalize(U_K)
    I_K = normalize(I_K)
    rtrain = load_npz("../../data/beer/Rtrain.npz")
    
    # Modify user preference matrix
    users = users_with_keyphrase_preference(U_K, keyphrase, top_users = top_users, norm = True) # 'raspberry'
    U_I = modify_item_history(rtrain, users, item_id = item_i)
    
    # make synthetic item1's keyphrase to all 0
    clear_item_keyphrase(I_K, item_id = item_i)
    
    # Get latent item similarity embedding
    modified_I_I = train(np.transpose(U_I))
    # predict
    explanation_scores1 = predict(I_K, 100, modified_I_I)
    explanation1 =  predict_pilot_explanation(explanation_scores1, top_keyphrase = 75)
    return list(explanation1[item_i]).index(keyphrase)

    
def evaluate_pilot_test_item(rtrain,U_K, I_K,keyphrase_list, top_users = 200, item_i = 0):
    # Get the average rank for item_i with keyphrase  
    res1 = 0
    for i in range(75):
        a = rank_in_prediction_item(rtrain, U_K, I_K, top_users = top_users, keyphrase = i, item_i = item_i)
        print "keyphrase", keyphrase_list[i], "'s rank is ", a
        res1+= a
    return res1/75

In [183]:
rank_in_prediction_item(rtrain, U_K, I_K, top_users = 200, keyphrase = 10, item_i = 0)

D:\Anaconda\envs\py27\lib\site-packages\ipykernel_launcher.py:23: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
D:\Anaconda\envs\py27\lib\site-packages\ipykernel_launcher.py:29: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
100%|██████████████████████████████████████████████████████████████████████████| 3668/3668 [00:00<00:00, 244535.51it/s]


5

In [187]:
evaluate_pilot_test_item(rtrain,U_K, I_K,keyphrases, top_users = 200, item_i = 0)

D:\Anaconda\envs\py27\lib\site-packages\ipykernel_launcher.py:23: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
D:\Anaconda\envs\py27\lib\site-packages\ipykernel_launcher.py:29: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
100%|██████████████████████████████████████████████████████████████████████████| 3668/3668 [00:00<00:00, 229249.54it/s]


keyphrase roast 's rank is  5


100%|██████████████████████████████████████████████████████████████████████████| 3668/3668 [00:00<00:00, 229249.54it/s]


keyphrase sweet 's rank is  0


100%|██████████████████████████████████████████████████████████████████████████| 3668/3668 [00:00<00:00, 229249.54it/s]


...

100%|██████████████████████████████████████████████████████████████████████████| 3668/3668 [00:00<00:00, 229249.54it/s]


keyphrase coriander 's rank is  40


100%|██████████████████████████████████████████████████████████████████████████| 3668/3668 [00:00<00:00, 229249.54it/s]


keyphrase pepper 's rank is  28


23.346666666666668

# TF-IDF pilot  test

In [112]:
keyphrases = pd.read_csv('../../data/beer/KeyPhrases.csv')["Phrases"].tolist()

In [122]:
save_npz('../../data/beer/U_K_tfidf',U_K_tfidf)
save_npz('../../data/beer/U_K_test_tfidf',U_K_test_tfidf)
save_npz('../../data/beer/I_K_tfidf',I_K_tfidf)
save_npz('../../data/beer/I_K_test_tfidf',I_K_test_tfidf)

In [125]:
def rank_in_prediction(rtrain, U_K, I_K, top_items = 200, keyphrase = [70],user_i = 0):
    """
    Get the rank for user_i with keyphrase
    TODO: modify so that no need to reload U_K,I_K
    """  
    U_K = load_npz('../../data/beer/U_K_tfidf.npz')
    rtrain = load_npz("../../data/beer/Rtrain.npz")
    I_K = load_npz('../../data/beer/I_K_tfidf.npz')
    
    # Modify user preference matrix
    items = item_associated_with_keyphrase(I_K, keyphrase, top_items = top_items) # 'raspberry'
    U_I = modify_user_preference(rtrain, items, user_id = 0)
    
    # make synthetic user1's keyphrase preference all 0
    clear_user_keyphrase(U_K, user_id = 0)
    
    # Get latent user similarity embedding
    modified_U_U = train(U_I)
    # predict
    explanation_scores1 = predict(U_K, 100, modified_U_U)
    explanation1 =  predict_pilot_explanation(explanation_scores1, top_keyphrase = 75)
    return list(explanation1[user_i]).index(keyphrase[0])
    
def evaluate_pilot_test(rtrain,U_K, I_K,keyphrase_list, top_items = 200, user_i = 0):
    # Get the average rank for user_i with keyphrase  
    res1 = 0
    for i in range(75):
        a = rank_in_prediction(rtrain, U_K, I_K, top_items = top_items, keyphrase = [i],user_i = user_i)
        print "keyphrase", keyphrase_list[i], "'s rank is ", a
        res1+= a
    return res1/75

In [133]:
evaluate_pilot_test(rtrain, U_K, I_K,keyphrases, top_items = 200, user_i = 0)

100%|██████████████████████████████████████████████████████████████████████████| 6370/6370 [00:00<00:00, 117963.18it/s]


keyphrase roast 's rank is  13


100%|██████████████████████████████████████████████████████████████████████████| 6370/6370 [00:00<00:00, 132708.06it/s]


keyphrase sweet 's rank is  0


100%|██████████████████████████████████████████████████████████████████████████| 6370/6370 [00:00<00:00, 148138.77it/s]


...

100%|██████████████████████████████████████████████████████████████████████████| 6370/6370 [00:00<00:00, 124901.67it/s]


keyphrase coriander 's rank is  66


100%|███████████████████████████████████████████████████████████████████████████| 6370/6370 [00:00<00:00, 96515.17it/s]


keyphrase pepper 's rank is  50


38.92